In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from pathlib import Path


In [3]:
#US-MMS data
path= "/home/jovyan/research_code/Transformers/temportal_fusion_transformers/data/CSIFMETEO/BDT_50_20/sorted_BDT_50_20_merged_1982_2021_US_MMS.parquet"
# Path to your dataset
parquet_path = Path(path)

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

# (1) Precompute historical plateaus (one‐time):
historical_vi = pd.read_csv("historical_vi.csv", parse_dates=["date"])
historical_vi["doy"] = historical_vi["date"].dt.dayofyear
seasonal_mean = historical_vi.groupby("doy")["vi"].mean().values[:365]
from scipy.ndimage import gaussian_filter1d
seasonal_mean_smooth = gaussian_filter1d(seasonal_mean, sigma=5)
VI_lower = seasonal_mean_smooth.min()
VI_upper = seasonal_mean_smooth.max()

# (2) On each daily “70‐day” window:
current_df = pd.read_csv("current_year_vi.csv", parse_dates=["date"])
current_df = current_df.sort_values("date").reset_index(drop=True)
current_df["doy"] = current_df["date"].dt.dayofyear

t = current_df["doy"].values             # 70 days
y = current_df["vi"].values               # 70 VI values

# (3) Define anchored logistic:
def anchored_logistic(t, b, t0):
    return VI_lower + (VI_upper - VI_lower) / (1 + np.exp(-b * (t - t0)))

# (4) ALILF‐style local iterative fit on spring side:
tol = 0.1          # days convergence
max_iter = 10
window_half_width = 20

# initialize bounds
t0_old = t.mean()  # start in the middle of your 70 days
for i in range(max_iter):
    # restrict data to window around t0_old
    win_lo = max(t.min(), t0_old - window_half_width)
    win_hi = min(t.max(), t0_old + window_half_width)
    mask = (t >= win_lo) & (t <= win_hi)
    t_win = t[mask]
    y_win = y[mask]

    # fit anchored logistic (a,c fixed)
    p0 = [0.1, t0_old]
    try:
        popt, _ = curve_fit(anchored_logistic, t_win, y_win, p0=p0, maxfev=10000)
    except RuntimeError:
        break
    b_fit, t0_new = popt

    if abs(t0_new - t0_old) < tol:
        break
    t0_old = t0_new

t0_spring = float(t0_new)
print(f"Spring green‐up roughly at DOY {t0_spring:.1f}")

# (5) (Optional) If your 70 days also cover senescence, repeat on the falling side:
mask_fall = t >= t0_spring
if mask_fall.sum() >= 5:  # need at least a few points
    t_fall = t[mask_fall]
    y_fall = y[mask_fall]
    # you could “invert” y_fall so that rising side is senescence:
    y_inv = VI_upper - y_fall
    # run the same iterative fit on (t_fall, y_inv):
    t0_old2 = t_fall.mean()
    for i in range(max_iter):
        win_lo2 = max(t_fall.min(), t0_old2 - window_half_width)
        win_hi2 = min(t_fall.max(), t0_old2 + window_half_width)
        mask2 = (t_fall >= win_lo2) & (t_fall <= win_hi2)
        t_win2 = t_fall[mask2]
        y_win2 = y_inv[mask2]
        p0_2 = [0.1, t0_old2]
        try:
            popt2, _ = curve_fit(anchored_logistic, t_win2, y_win2, p0=p0_2, maxfev=10000)
        except RuntimeError:
            break
        b_fit2, t0_new2 = popt2
        if abs(t0_new2 - t0_old2) < tol:
            break
        t0_old2 = t0_new2
    t0_autumn = float(t0_new2)
    print(f"Autumn senescence roughly at DOY {t0_autumn:.1f}")
else:
    t0_autumn = None
    print("Not enough data after spring to estimate autumn.")

# (6) As a fallback, you could do second‐derivative on the smoothed VI:
vi_smoothed = savgol_filter(y, 11, 2)
d1 = np.gradient(vi_smoothed, t)
d2 = np.gradient(d1, t)
idx_spring2 = np.argmax(d2[:50])        # first half of 70 days
t0_spring2 = t[idx_spring2]
if mask_fall.sum() >= 5:
    idx_autumn2 = idx_spring2 + np.argmin(d2[idx_spring2:])
    t0_autumn2 = t[idx_autumn2]
else:
    t0_autumn2 = None

# Compare results:
print("ALILF‐anchored spring t0:", t0_spring)
print("Second‐derivative spring t0:", t0_spring2)
if t0_autumn is not None:
    print("ALILF‐anchored autumn t0:", t0_autumn)
    print("Second‐derivative autumn t0:", t0_autumn2)
